In [1]:
import pandas as pd

In [8]:
results_filepath_prefix = '../results/'
result_filename = 'test_channels.csv'
result_filepath = results_filepath_prefix + result_filename

result_df = pd.read_csv(result_filepath)
result_df.head()


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,video_id,rank,component,bot_id,phase,level,time,channel_id,parent_video_id,political_label
0,0,0,0,9CdY7t5VKIY,0,homepage,sean.carter.99.test,create,0,2022-03-16 15:52:11.725290,UC5O114-PQNYkurlTg6hekZw,NaN,C
1,1,1,1,aOcAqCvH66Q,1,homepage,sean.carter.99.test,create,0,2022-03-16 15:52:11.734259,UCSFEyis_hTphbn3suigLCxg,NaN,Other
2,2,2,2,hg-HbRIk0xM,2,homepage,sean.carter.99.test,create,0,2022-03-16 15:52:11.742567,UCBZN6dwpyWl_WoRxdsLiErg,NaN,Other
3,3,3,3,KM36zdNUzZk,3,homepage,sean.carter.99.test,create,0,2022-03-16 15:52:11.750553,UCXnFys9ZXBE0uyDhKHUi-dA,NaN,Other
4,4,4,4,oznAFplzZJA,4,homepage,sean.carter.99.test,create,0,2022-03-16 15:52:11.759086,UC7sPt5ZKCORuvggr_Sj36NA,NaN,Other


In [9]:
categories = ['FR','R', 'AW', 'C', 'L', 'FL', 'Other']
scores = [1,0.5,0.5,0,-0.5,-1,0]
h_label_map = dict(zip(categories, scores))

In [10]:
# Copied from old code (analysis/assign_labels.ipynb)
# labels = list of political labels
# label_map = map from political labels to numerical score
# see Tomlein et al. (2021) for description of formula
# or the Juneja paper
def serp_ms(labels, label_map):
    # No recommendations
    if len(labels)==1:
        return 0
    
    num = 0
    n = len(labels)
    r = 1
    for label in labels[1:]:
        num += label_map[label] * (n - r + 1)
        r += 1
    den = n * (n+1) / 2.0
    if den == 0: return 0
    return num / den

In [11]:
def h_serp_ms(labels):
    return serp_ms(labels, h_label_map)

In [15]:
serp_ms_scores = result_df.groupby(['phase','level', 'component'])['political_label'].agg([h_serp_ms])
serp_ms_scores

h_serp_ms
phase  level component           
burst  0     homepage   -0.054545
             videopage   0.054545
       1     homepage   -0.063636
             videopage  -0.045455
       2     homepage    0.072727
             videopage  -0.163636
       3     homepage    0.000000
             videopage  -0.090909
       4     homepage   -0.036364
             videopage  -0.090909
create 0     homepage    0.000000
             videopage  -0.018182
       1     homepage    0.000000
             videopage   0.154545
       2     homepage    0.000000
             videopage  -0.009091
       3     homepage    0.145455
             videopage   0.090909
       4     homepage   -0.063636
             videopage  -0.072727

In [16]:
serp_ms_scores.to_csv(results_filepath_prefix + 'test_channels_serp.csv', index=True)